In [1]:
import pandas as pd
import re

In [2]:
keys = pd.read_csv('new_keys.csv')
data = pd.read_csv('Suburb_House_final.csv')

In [3]:

housep_df = data[['Median House  Price - 2019', 'Unnamed: 11']]
housep_df = housep_df.iloc[1:]


adjus_hp_df = pd.DataFrame({'locality': housep_df['Median House  Price - 2019'].iloc[1:], 
                            'Median House Price - 2019': housep_df['Unnamed: 11'].iloc[1:]}).reset_index(drop=True)
print(adjus_hp_df)

print(adjus_hp_df['Median House Price - 2019'])

         locality Median House Price - 2019
0      ABBOTSFORD                   1030000
1      ABERFELDIE                   1390000
2         AINTREE                    610000
3    AIREYS INLET                    985000
4    AIRPORT WEST                    790000
..            ...                       ...
798        YARRAM                    240000
799    YARRAVILLE                    971000
800    YARRAWONGA                    375000
801           YEA                    435500
802        YINNAR                    255000

[803 rows x 2 columns]
0      1030000
1      1390000
2       610000
3       985000
4       790000
        ...   
798     240000
799     971000
800     375000
801     435500
802     255000
Name: Median House Price - 2019, Length: 803, dtype: object


In [4]:
keys_df = pd.DataFrame({'Suburb':keys['Suburb'], 'LGA names':keys['LGA names']})
keys_dict = keys_df.set_index('Suburb').T.to_dict('list')
keys_dict

{'ABBOTSFORD': ['YARRA'],
 'ABERFELDIE': ['MOONEE VALLEY'],
 'AIREYS INLET': ['SURF COAST'],
 'AIRLY': ['WELLINGTON'],
 'AIRPORT WEST': ['MOONEE VALLEY'],
 'ALBANVALE': ['BRIMBANK'],
 'ALBERT PARK': ['PORT PHILLIP'],
 'ALBERTON': ['WELLINGTON'],
 'ALBION': ['BRIMBANK'],
 'ALEXANDRA': ['MURRINDINDI'],
 'ALFREDTON': ['BALLARAT'],
 'ALLANSFORD': ['WARRNAMBOOL'],
 'ALPHINGTON': ['YARRA'],
 'ALTONA': ['HOBSONS BAY'],
 'ALTONA MEADOWS': ['HOBSONS BAY'],
 'ALTONA NORTH': ['HOBSONS BAY'],
 'ALVIE': ['COLAC OTWAY'],
 'AMPHITHEATRE': ['PYRENEES'],
 'ANAKIE': ['GREATER GEELONG'],
 'ANGLESEA': ['SURF COAST'],
 'APOLLO BAY': ['COLAC OTWAY'],
 'APSLEY': ['WEST WIMMERA'],
 'ARARAT': ['ARARAT'],
 'ARDEER': ['BRIMBANK'],
 'ARDMONA': ['GREATER SHEPPARTON'],
 'ARMADALE': ['STONNINGTON'],
 'ARMSTRONG CREEK': ['GREATER GEELONG'],
 'ARTHURS CREEK': ['NILLUMBIK'],
 'ASCOT VALE': ['MOONEE VALLEY'],
 'ASHBURTON': ['BOROONDARA'],
 'ASHWOOD': ['MONASH'],
 'ASPENDALE': ['KINGSTON'],
 'ASPENDALE GARDENS': ['KINGST

In [5]:
adju_loctolga = adjus_hp_df['locality']
adju_loctolga


0        ABBOTSFORD
1        ABERFELDIE
2           AINTREE
3      AIREYS INLET
4      AIRPORT WEST
           ...     
798          YARRAM
799      YARRAVILLE
800      YARRAWONGA
801             YEA
802          YINNAR
Name: locality, Length: 803, dtype: object

In [6]:
count = 0
for i in range(len(adju_loctolga)):
    if adju_loctolga[i] in keys_dict:
        adju_loctolga[i] = keys_dict[adju_loctolga[i]][0]
        count+=1
    else:
        adju_loctolga[i] += ' *no lga*'
print(count)
adju_loctolga

624


0                 YARRA
1         MOONEE VALLEY
2      AINTREE *no lga*
3            SURF COAST
4         MOONEE VALLEY
             ...       
798          WELLINGTON
799         MARIBYRNONG
800               MOIRA
801         MURRINDINDI
802             LATROBE
Name: locality, Length: 803, dtype: object

In [7]:
process_hp_df = pd.DataFrame({'LGA': adju_loctolga, 'Median House Price - 2019':adjus_hp_df['Median House Price - 2019']} )

In [8]:
process_hp_df

,LGA,Median House Price - 2019
0,YARRA,1030000
1,MOONEE VALLEY,1390000
2,AINTREE *no lga*,610000
3,SURF COAST,985000
4,MOONEE VALLEY,790000
...,...,...
798,WELLINGTON,240000
799,MARIBYRNONG,971000
800,MOIRA,375000
801,MURRINDINDI,435500


In [9]:
process_hp_df.to_csv('Median_House_Price-2019.csv', index=False)

In [10]:
# found average price, total_hp is the sum of the suburbs houseprice in the lga
lga_hprice_df = pd.read_csv("lga_fill_Median_House_Price-2019.csv")
lga_freq = list(lga_hprice_df['LGA'].value_counts().sort_index(ascending=True))
lga_hprice_df['Median House Price - 2019'] = pd.to_numeric(lga_hprice_df['Median House Price - 2019'])
lga_hprice_median = lga_hprice_df.groupby('LGA')['Median House Price - 2019'].median()
lga_hprice_df = lga_hprice_df.groupby('LGA')['Median House Price - 2019'].sum().to_frame().reset_index()
print(lga_hprice_median)
lga_hprice_df['Number of Suburbs in LGA'] = lga_freq
total_hp = lga_hprice_df['Median House Price - 2019']
lga_hprice_df['Average Pouse Price'] = lga_hprice_df['Median House Price - 2019'] / lga_hprice_df['Number of Suburbs in LGA']
lga_hprice_df['Average Pouse Price'] = lga_hprice_df['Average Pouse Price'].round(3)
lga_hprice_df['Median Pouse Price'] = list(lga_hprice_median)
lga_hprice_df = lga_hprice_df.drop(columns='Median House Price - 2019')
lga_hprice_df

LGA
ALPINE           460000
ARARAT           163250
BALLARAT         416000
BANYULE          880500
BASS COAST       495500
                 ...   
WODONGA          395000
WYNDHAM          565000
YARRA           1352500
YARRA RANGES     642500
YARRIAMBIACK     114500
Name: Median House Price - 2019, Length: 79, dtype: int64


,LGA,Number of Suburbs in LGA,Average Pouse Price,Median Pouse Price
0,ALPINE,7,443428.571,460000
1,ARARAT,2,163250.000,163250
2,BALLARAT,30,431016.667,416000
3,BANYULE,18,991750.000,880500
4,BASS COAST,23,469630.435,495500
...,...,...,...,...
74,WODONGA,5,389500.000,395000
75,WYNDHAM,10,583350.000,565000
76,YARRA,11,1370272.727,1352500
77,YARRA RANGES,37,633351.351,642500


In [11]:
lga_hprice_df.to_csv('LGA_House_Price-2019+stats.csv', index=False)